In [1]:
%cd ../..

/home/zaccharie/workspace/understanding-unets


In [2]:
# this just to make sure we are using only on CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
import copy
import time

import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

from learning_wavelets.datasets import im_dataset_bsd68
from learning_wavelets.utils.metrics import metrics_original_from_ds, metrics_bm3d_from_ds

In [4]:
np.random.seed(0)

In [5]:
# noise_stds = [5, 15, 20, 25, 30, 50, 75]
noise_stds = [5, 15, 20]

In [6]:
noise_std_metrics = {}
n_samples = 3
for noise_std in tqdm_notebook(noise_stds, 'Noise stds'):
    metrics = []
    im_ds = im_dataset_bsd68(
        mode='testing', 
        batch_size=1, 
        patch_size=None, 
        noise_std=noise_std, 
        return_noise_level=False,
        n_pooling=5,
        n_samples=n_samples,
    )
    metrics.append(('original', metrics_original_from_ds(im_ds)))
        

    im_ds = im_dataset_bsd68(
        mode='testing', 
        batch_size=1, 
        patch_size=None, 
        noise_std=noise_std, 
        return_noise_level=False,
        n_pooling=5,
        n_samples=n_samples,
    )
    metrics.append(('bm3d', metrics_bm3d_from_ds(im_ds, noise_std=noise_std)))
    noise_std_metrics[noise_std] = metrics

> /home/zaccharie/workspace/understanding-unets/learning_wavelets/utils/metrics.py(57)metrics_bm3d_from_ds()
     56     pred_and_gt_shape = [
---> 57         (
     58             bm3d.bm3d(

ipdb> c


> /home/zaccharie/workspace/understanding-unets/learning_wavelets/utils/metrics.py(57)metrics_bm3d_from_ds()
     56     pred_and_gt_shape = [
---> 57         (
     58             bm3d.bm3d(

ipdb> c


> /home/zaccharie/workspace/understanding-unets/learning_wavelets/utils/metrics.py(57)metrics_bm3d_from_ds()
     56     pred_and_gt_shape = [
---> 57         (
     58             bm3d.bm3d(

ipdb> c


In [7]:
%debug

E0225 09:08:26.863719 139959247701760 interactiveshell.py:1178] No traceback has been produced, nothing to debug.


In [8]:
noise_std_metrics

{5: [('original', PSNR = 34.15 +/- 0.02193 SSIM = 0.8648 +/- 0.06844),
  ('bm3d', PSNR = 6.018 +/- 0.00183 SSIM = -0.343 +/- 0.3291)],
 15: [('original', PSNR = 24.61 +/- 0.01594 SSIM = 0.5177 +/- 0.08687),
  ('bm3d', PSNR = 6.01 +/- 0.004426 SSIM = -0.31 +/- 0.3498)],
 20: [('original', PSNR = 22.11 +/- 0.02579 SSIM = 0.4143 +/- 0.05454),
  ('bm3d', PSNR = 6.002 +/- 0.008097 SSIM = -0.2979 +/- 0.3628)]}

In [9]:
# PSNR table
psnr_metrics_table = pd.DataFrame(
    columns=['noise_std'] + ['original', 'wavelets_24', 'bm3d'],
)
for i, (noise_std, metrics) in enumerate(noise_std_metrics.items()):
    psnr_metrics_table.loc[i, 'noise_std'] = noise_std
    for name, m in metrics:
        psnr_metrics_table.loc[i, name] = "{mean:.4} ({std:.4})".format(
            mean=m.metrics['PSNR'].mean(), 
            std=m.metrics['PSNR'].stddev(),
        )
psnr_metrics_table

,noise_std,original,wavelets_24,bm3d
0,5,34.15 (0.01096),NaN,6.018 (0.0009148)
1,15,24.61 (0.007971),NaN,6.01 (0.002213)
2,20,22.11 (0.0129),NaN,6.002 (0.004049)


In [10]:
# SSIM table
ssim_metrics_table = pd.DataFrame(
    columns=['noise_std'] + ['original', 'wavelets_24', 'bm3d'],
)
for i, (noise_std, metrics) in enumerate(noise_std_metrics.items()):
    ssim_metrics_table.loc[i, 'noise_std'] = noise_std
    for name, m in metrics:
        ssim_metrics_table.loc[i, name] = "{mean:.4} ({std:.4})".format(
            mean=m.metrics['SSIM'].mean(), 
            std=m.metrics['SSIM'].stddev(),
        )
ssim_metrics_table

,noise_std,original,wavelets_24,bm3d
0,5,0.8648 (0.03422),NaN,-0.343 (0.1645)
1,15,0.5177 (0.04344),NaN,-0.31 (0.1749)
2,20,0.4143 (0.02727),NaN,-0.2979 (0.1814)
